In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
import numpy as np 
import pandas as pd 

import os
import nltk
import gensim
nltk.download('stopwords')
!pip install smart_open
import smart_open
from gensim.test.utils import datapath, get_tmpfile
from smart_open import open

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
from gensim.models import KeyedVectors
from gensim.models import Word2Vec
from gensim.scripts.glove2word2vec import glove2word2vec
import tweepy
import requests
import tensorflow 

In [ ]:
glove_file = datapath('/content/drive/My Drive/data/glove.6B.100d.txt')
tmp_file = get_tmpfile("test_word2vec.txt")
_ = glove2word2vec(glove_file, tmp_file)
embeddings = KeyedVectors.load_word2vec_format(tmp_file)
print(embeddings)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [ ]:
filepath = "/content/drive/My Drive/data/glove.6B.100d.txt"
with open (filepath, "r") as f:
  r = f.read()
  print(len(r))

347104530


In [ ]:
data = pd.read_csv('/content/drive/My Drive/data/final_output1.csv',encoding='latin1',usecols=['text_without_stopwords','type'])
data.head()

,type,text_without_stopwords
0,bias,print pay back money plus interest entire fami...
1,bias,attorney general loretta lynch plead fifth ba...
2,bias,red state fox news sunday reported morning ant...
3,bias,email kayla mueller prisoner tortured isis cha...
4,bias,email healthcare reform make america great sin...


In [ ]:
data.rename(columns={0:'type', 1:'text'}, inplace=True)
data.head()

,type,text_without_stopwords
0,bias,print pay back money plus interest entire fami...
1,bias,attorney general loretta lynch plead fifth ba...
2,bias,red state fox news sunday reported morning ant...
3,bias,email kayla mueller prisoner tortured isis cha...
4,bias,email healthcare reform make america great sin...


In [ ]:
data.loc[0, 'text_without_stopwords']

'print pay back money plus interest entire family everyone came need deported asap take two years bust go another group stealing government taxpayers group somalis stole four million government benefits months weve reported numerous cases like one muslim refugeesimmigrants commit fraud scamming systemits way control related'

In [ ]:
docs_vectors = pd.DataFrame() 
stopwords = nltk.corpus.stopwords.words('english') 
for doc in data['text_without_stopwords'].str.lower().str.replace('[^a-z ]', ''): 
    temp = pd.DataFrame()  
    for word in str(doc).split(' '): 
        try:
            word_vec = embeddings[word] 
            temp = temp.append(pd.Series(word_vec), ignore_index = True) 
        except:
            pass
    doc_vector = temp.mean() 
    docs_vectors = docs_vectors.append(doc_vector, ignore_index = True) 
docs_vectors.shape
print(docs_vectors)

             0         1         2   ...        97        98        99
0      0.105690  0.319595  0.121895  ...  0.010271  0.474572 -0.053729
1      0.011914 -0.008603  0.230136  ...  0.006763  0.279298 -0.038708
2     -0.049037 -0.015963  0.240281  ... -0.159232  0.299037  0.143670
3     -0.012512  0.087773  0.170533  ... -0.258531 -0.018412  0.116490
4     -0.063122  0.210538  0.234165  ... -0.105908  0.507807 -0.020383
...         ...       ...       ...  ...       ...       ...       ...
15380 -0.040383 -0.213200  0.055937  ... -1.392700 -0.015867  0.113110
15381 -0.040383 -0.213200  0.055937  ... -1.392700 -0.015867  0.113110
15382 -0.040383 -0.213200  0.055937  ... -1.392700 -0.015867  0.113110
15383 -0.040383 -0.213200  0.055937  ... -1.392700 -0.015867  0.113110
15384 -0.040383 -0.213200  0.055937  ... -1.392700 -0.015867  0.113110

[15385 rows x 100 columns]


In [ ]:
pd.isnull(docs_vectors).sum().sum()

200

In [ ]:
docs_vectors['text_without_stopwords'] = data['type']
docs_vectors = docs_vectors.dropna()

In [ ]:
from sklearn.model_selection import train_test_split

train_x, test_x, train_y, test_y = train_test_split(docs_vectors.drop('text_without_stopwords', axis = 1),
                                                   docs_vectors['text_without_stopwords'],
                                                   test_size = 0.4,
                                                   random_state = 1)
train_x.shape, train_y.shape, test_x.shape, test_y.shape

((1257, 100), (1257,), (838, 100), (838,))

Adaboost:

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
model = AdaBoostClassifier(n_estimators=100, random_state = 1)
model.fit(train_x, train_y)
test_pred = model.predict(test_x)
from sklearn.metrics import accuracy_score
accuracy_score(test_y, test_pred)

0.3238095238095238

Random Forest:

In [ ]:
from sklearn.ensemble import RandomForestClassifier
Rando= RandomForestClassifier(n_estimators=100)
Rando.fit(train_x,train_y)
y_pred1=Rando.predict(test_x)
accuracy_score(test_y,y_pred1)

0.49047619047619045

SVM:

In [ ]:
from sklearn.svm import SVC
classifier=SVC(kernel='linear',random_state=0)
classifier.fit(train_x,train_y)
y_pred=classifier.predict(test_x)
accuracy_score(test_y,y_pred)

0.42857142857142855

Logistic Regression:

In [ ]:
from sklearn.linear_model import LogisticRegression
logreg=LogisticRegression(C=1e5)
logreg.fit(train_x,train_y)
y_pred2=logreg.predict(test_x)
accuracy_score(test_y,y_pred2)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.44761904761904764

Decision Tree Classifier:

In [ ]:
from sklearn.tree import DecisionTreeClassifier
svc = DecisionTreeClassifier().fit(train_x, train_y)
y_pred2 = svc.predict(test_x)
accuracy_score(test_y,y_pred2)

0.45714285714285713

K fold:

In [ ]:
from sklearn import model_selection
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold

kfold = model_selection.KFold(n_splits=10,random_state=100)
model_kfold = RandomForestClassifier(n_estimators=100)
results_kfold = model_selection.cross_val_score(model_kfold,docs_vectors.drop('text_without_stopwords', axis = 1),
                                                   docs_vectors['text_without_stopwords'],cv=kfold)
print("Accuracy: %.2f%%" % (results_kfold.mean()*100.0)) 

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


Accuracy: 20.41%


Stratified K fold:

In [ ]:
skfold = StratifiedKFold(n_splits=3, random_state=100)
model_skfold = RandomForestClassifier(n_estimators=100)
results_skfold = model_selection.cross_val_score(model_skfold,docs_vectors.drop('text_without_stopwords', axis = 1),
                                                   docs_vectors['text_without_stopwords'], cv=skfold)
print("Accuracy: %.2f%%" % (results_skfold.mean()*100.0))

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


Accuracy: 28.16%
